In [1]:
import pandas as pd

In [2]:
movies = pd.read_json("movies.json")
reviews = pd.read_json("reviews.json")
reviews.head()

,user_id,item_id,rating,timestamp
0,0,50,5,1997-12-04 15:55:49
1,0,172,5,1997-12-04 15:55:49
2,0,133,1,1997-12-04 15:55:49
3,196,242,3,1997-12-04 15:55:49
4,186,302,3,1998-04-04 19:22:22


In [3]:
reviews.drop(['timestamp'], inplace=True, axis=1)

In [4]:
mostReviewed = 3
topN = reviews.groupby('item_id').size()

In [5]:
topN = topN.sort_values(ascending=False).reset_index()[:mostReviewed]
topN

,item_id,0
0,50,583
1,258,507
2,100,506


In [6]:
top3DF = reviews[reviews['item_id'].isin(topN['item_id'])]

In [7]:
pivotDF = top3DF.pivot(index='user_id', columns='item_id', values='rating')

In [8]:
pivotDF = pivotDF.dropna(axis=0, how="all")

In [9]:
pivotDF = pivotDF.fillna(-1)

In [10]:
pivotDF.head()

item_id,50,100,258
user_id,,,
0,5.0,-1.0,-1.0
1,5.0,5.0,5.0
2,5.0,5.0,3.0
3,-1.0,-1.0,2.0
4,5.0,-1.0,5.0


In [11]:
from sklearn.cluster import MiniBatchKMeans

In [12]:
kb = MiniBatchKMeans(n_clusters=2, random_state=0, batch_size=6)

In [13]:
kb = kb.fit(pivotDF)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


In [14]:
kb.cluster_centers_

array([[3.88235294, 2.61764706, 3.82352941],
       [1.47368421, 2.02631579, 0.57894737]])

In [15]:
from sklearn.metrics import silhouette_score

In [16]:
ss = silhouette_score(pivotDF, kb.labels_)

In [17]:
ss

0.28421773323138216

In [19]:
scores = {}
for i in range(2, 10):
    kb = MiniBatchKMeans(n_clusters=i, random_state=0, batch_size=6)
    kb = kb.fit(pivotDF)
    if(len(set(kb.labels_)) > 1):
        ss = silhouette_score(pivotDF, kb.labels_)
        scores[i] = ss

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\Prog

In [20]:
scores

{2: 0.28421773323138216,
 3: 0.37564170106287786,
 4: 0.41966544617439666,
 5: 0.4928480710445517,
 6: 0.580962944977297,
 7: 0.5792052104909716,
 8: 0.6205231475924143,
 9: 0.4634056548041324}

In [21]:
kb = MiniBatchKMeans(n_clusters=8, random_state=0, batch_size=6)
kb = kb.fit(pivotDF)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1043: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 5120 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


In [22]:
pivotDF['Cluster'] = kb.labels_

In [24]:
pivotDF.head(10)

item_id,50,100,258,Cluster
user_id,,,,
0,5.0,-1.0,-1.0,7
1,5.0,5.0,5.0,6
2,5.0,5.0,3.0,6
3,-1.0,-1.0,2.0,5
4,5.0,-1.0,5.0,3
5,4.0,5.0,-1.0,0
6,4.0,5.0,2.0,6
7,5.0,5.0,4.0,6
8,5.0,-1.0,5.0,3


In [26]:
user8 = reviews[reviews['user_id'] == 8]

In [27]:
user8

,user_id,item_id,rating
93,8,338,4
518,8,550,3
684,8,22,5
1089,8,50,5
3840,8,182,5
4641,8,79,4
6026,8,294,3
6117,8,457,1
6122,8,385,1
7033,8,89,4


In [28]:
user4 = reviews[reviews['user_id'] == 4]

In [29]:
user4

,user_id,item_id,rating
1249,4,264,3
1328,4,303,5
2201,4,361,5
2523,4,357,4
3272,4,260,4
5952,4,356,3
12135,4,294,5
13875,4,288,4
16284,4,50,5
18908,4,354,5


In [30]:
user8[user8['item_id'] == 303]

,user_id,item_id,rating


In [31]:
movies[movies['item_id'] == 303]

,item_id,title
302,303,Ulee's Gold (1997)
